recoding.ipynb

Recode data for ACN5314 Group Project, CRCNS pfc-6 data.

I am using Python since Athul's CMPUtils is broken for Mac and he did not provide any solution... I will make sure that this output is compatible with Julia.

Last edit:  09 MAR 2022, CDR ||
Created:    08 MAR 2022, CDR

In [1]:
import pandas as pd

In [20]:
# DECLARE GLOBALS


# Folder containing all session data
dataFolder = '../pfc-6/mPFC_Data'

# column headers (see pfc-6 documentation for details)
columnHeaders = {
    "Behavior"  :    ['trialStart', 'trialEnd', 'rule', 'correct', 'direction','lightPos'],
    "SpikeData" :   ['spikeTime','cellID'],
    "CellType"  :   ['cellType'],
    "WakeEpoch"      :   ['behaviorStart','behaviorEnd'],
    "SwsPost"   :   ['behaviorStart','behaviorEnd'],
    "SwsPre"    :   ['behaviorStart','behaviorEnd'],
    "Pos"       :   ['time','x','y']
}

In [3]:
# get all the files in dataFolder
dir = os.listdir(dataFolder)

# alphabetical sort
dir = sorted(dir)

# remove os files like ds_store; all the data folders are numeric (session ID: IDMMDD - ID: animal number, MM: month, DD: day)
for q in dir:
    if not q.isnumeric():
        dir.remove(q)

# dir

In [17]:
def getDfFromFile(dfId, sessionId):
    name = "%s_%s"%(sessionId,dfId)
    fileName = "%s/%s/%s_%s.dat"%(dataFolder,sessionId,sessionId,dfId)

    cols = columnHeaders.get(dfId)
    assert cols != None, "Invalid dataframe Id"

    df = pd.read_fwf(fileName, name=cols, header=None)
    df.name = dfId
    df.columns = cols

    return df

In [32]:
# todo: cycle through all sessions
sessionId = '150628' # 6dig; IDMMDD - animal ID, month, day

# get all the dataframes we have for this session
behDf       = getDfFromFile("Behavior", sessionId)
spikeDf     = getDfFromFile("SpikeData", sessionId)
cellDf      = getDfFromFile("CellType", sessionId)
wakeDf      = getDfFromFile("WakeEpoch", sessionId)
swsPreDf    = getDfFromFile("SwsPre", sessionId)
swsPostDf   = getDfFromFile("SwsPost", sessionId)
posDf       = getDfFromFile("Pos", sessionId)

In [28]:
print("BEHAVIOR:")
print(behDf.head())

print("SPIKE:")
print(spikeDf.head())

print("CELL TYPE:")
print(cellDf.head())

BEHAVIOR:
   trialStart   trialEnd  rule  correct  direction  lightPos
0   1512078.7  1516415.3   1.0      0.0        1.0       1.0
1   1541400.8  1558246.9   1.0      0.0        1.0       0.0
2   1606616.7  1613722.0   1.0      0.0        1.0       1.0
3   1662625.6  1667729.4   1.0      1.0        0.0       0.0
4   1728441.9  1732078.0   1.0      0.0        1.0       0.0
SPIKE:
   spikeTime  cellID
0      323.2     1.0
1     1035.2     1.0
2     5830.2     1.0
3     7425.0     1.0
4     8025.3     1.0
CELL TYPE:
   cellType
0         1
1         2
2         2
3         2
4         2


In [33]:
print("WAKE EPOCH:")
print(wakeDf.head())

print("SWS PRE:")
print(swsPreDf.head())

print("SWS POST:")
print(swsPostDf.head())

WAKE EPOCH:
   behaviorStart  behaviorEnd
0      1494765.7    3191011.7
SWS PRE:
   behaviorStart  behaviorEnd
0        67016.2     123996.6
1       573005.2    1150011.3
2      1213993.5    1304015.2
SWS POST:
   behaviorStart  behaviorEnd
0      3603022.4    3751000.7
1      3922029.7    3990014.3
2      4061001.1    4506003.4
